In [0]:
%cd "drive/My Drive/darknet"

In [0]:
%ls

In [0]:
%ls backup

# Cropper armoire centrale

#### Utils:

In [0]:
!pip install -q numpy matplotlib tqdm opencv-python

In [0]:
import os
import glob
import time
from typing import List

import cv2
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt


class Cropper:
    """
    Load model from config and weights file

    cfg:
        - Path to config file
    weights:
        - Path to weights file
    thresholds:
        - [0] is default threshold and [1] threshold used
        if nothing is detected with the first
    """
    def __init__(self, cfg: str, weights: str, thresholds: List[float] = [0.5, 0.4]):
        self.cfg = cfg
        self.weights = weights
        self.CONFTHRESH = thresholds  # set to 0.3 if not working
        self.NMSTHRESH = 0.3
        self.model = cv2.dnn.readNetFromDarknet(self.cfg, self.weights)
        print("🚀 Model loaded")

    
    def __show_dif(self, img, crop):
        """
        Returns subplots with original and cropped images
        """
        _, ax = plt.subplots(1, 2, figsize=(8, 4))
        ax[0].set_title('Predicted image')
        ax[0].imshow(img)
        ax[1].set_title('Cropped image')
        ax[1].imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        plt.show()

    def __crop(self, img):
        """
        Get prediction and return cropped image
        """
        (H, W) = img.shape[:2]
        layer_names = self.model.getLayerNames()
        layer_names = [
            layer_names[i[0] - 1] for i in self.model.getUnconnectedOutLayers()
        ]

        blob = cv2.dnn.blobFromImage(
            img,
            1 / 255.0,
            (416, 416),
            swapRB=False,
            crop=False
        )

        self.model.setInput(blob)
        layer_outputs = self.model.forward(layer_names)

        boxes = []
        confidences = []
        classes_ids = []
        thresh = 0

        for output in layer_outputs:

            for detected in output:

                scores = detected[5:]
                class_id = np.argmax(scores)
                conf = scores[class_id]
                if conf > self.CONFTHRESH[0]:
                    box = detected[0:4] * np.array([W, H, W, H])
                    (x_center, y_center, width, height) = box.astype("int")

                    x = int(x_center - (width / 2))
                    y = int(y_center - (height / 2))

                    boxes.append(
                        [x, y, int(width), int(height)]
                    )
                    confidences.append(float(conf))
                    classes_ids.append(class_id)
                    thresh = self.CONFTHRESH[0]

                elif self.CONFTHRESH[1] < conf < self.CONFTHRESH[0]:
                    box = detected[0:4] * np.array([W, H, W, H])
                    (x_center, y_center, width, height) = box.astype("int")

                    x = int(x_center - (width / 2))
                    y = int(y_center - (height / 2))

                    boxes.append(
                        [x, y, int(width), int(height)]
                    )
                    confidences.append(float(conf))
                    classes_ids.append(class_id)
                    thresh = self.CONFTHRESH[1]

                else:
                    cropped_image = None

        print(f"\n🔢 Threshold used: {thresh}")
        idxs = cv2.dnn.NMSBoxes(
            boxes,
            confidences,
            thresh,
            self.NMSTHRESH
        )

        if len(idxs) > 0:

            for i in idxs.flatten():

                (x, y) = (max(0, boxes[i][0]), max(0, boxes[i][1]))
                (w, h) = (boxes[i][2], boxes[i][3])
                cropped_image = img[y:y+h, x:x+w]

        return cropped_image


    def predict(self, item: str, length: int = -1, output_folder: str = 'predictions/'):
        """
        Predict one image or a folder of images and save cropped result

        item:
            - Path to image or folder of images
        length:
            - Select *length* images from folder [default: -1]
        output_folder:
            - Folder to save result [default: 'predictions/']
        """
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        if '.jpg' in item:

            image = plt.imread(item)
            # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            print("💡 Making prediction")
            start = time.time()
            output_image = self.__crop(img=image)
            if output_image is not None:
                print(f"⏱️ Predicted in {time.time() - start:.2f} sec")

                output_path = os.path.join(output_folder, os.path.basename(item))

                output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

                cv2.imwrite(output_path, output_image)

                print(f"💾 Saved to {output_path}")
                self.__show_dif(image, output_image)
            else:
                print(f"😥 Found nothing on {item}")

        else:
            not_detected = []
            items = glob.glob(
                os.path.join(item, '*.jpg')
            )[:length] if length > 0 else glob.glob(os.path.join(item, '*.jpg'))
            preds = [
                os.path.basename(i) for i in glob.glob(
                    os.path.join(output_folder, '*.jpg')
                )
            ]

            for i in tqdm(items, desc="💡 Making predictions"):
                if os.path.basename(i) not in preds: 
                    image = plt.imread(i)
                    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                    start = time.time()
                    output_image = self.__crop(img=image)
                    if output_image is not None:
                        print(f"⏱️ Predicted in {time.time() - start:.2f} sec")

                        output_path = os.path.join(output_folder, os.path.basename(i))

                        output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

                        cv2.imwrite(output_path, output_image)

                        print(f"💾 Saved to {output_path}")
                        self.__show_dif(image, output_image)
                    else:
                        print(f"😥 Found nothing on {i}")
                        not_detected.append(i)
                else:
                    pass
            print(f"\n❌ There are {len(not_detected)} images where nothing was detected:")
            for nd in not_detected:
                print('\t' + nd)

#### Run:

In [0]:
del crop

In [0]:
crop = Cropper(
    cfg='cfg/custom/tiny-armoire.cfg',
    weights='backup/tiny-armoire_best.weights',
    thresholds=[0.5, 0.3]
)

In [0]:
crop.predict("../test/", output_folder='../predictions')

In [0]:
# !rm predictions/*.jpg

not detected: 20200120_160657.jpg & C_05.jpg

# Cropper en trois zones: gauche - milieu - droite

#### Utils:

In [0]:
import os
import glob

import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt


def split_images(input_folder: str, output_folder: str = './', method: str = 'plt'):
    """
    Split folder of images in three parts

    input_folder:
        - Path to images folder
    output_folder:
        - Path to output folder
    method:
        - Using plt or cv2 method
    """

    imgs = glob.glob(os.path.join(input_folder, '*.jpg'))

    if method == 'plt':
        for i in tqdm(imgs, desc="✂️ Splitting images"):
            img = plt.imread(i)
            h, w, d = img.shape
            plt.imsave(
                f"{os.path.join(output_folder, 'a/')}{os.path.basename(i)}",
                img[:, :round(w*0.4)]
            )
            plt.imsave(
                f"{os.path.join(output_folder, 'b/')}{os.path.basename(i)}",
                img[:, round(w*0.4):round(w*0.6)]
            )
            plt.imsave(
                f"{os.path.join(output_folder, 'c/')}{os.path.basename(i)}",
                img[:, round(w*0.6):]
            )

    elif method == 'cv2':
        for i in tqdm(imgs, desc="✂️ Splitting images"):
            img = cv2.imread(i)
            h, w, d = img.shape
            cv2.imwrite(
                f"{os.path.join(output_folder, 'a/')}{os.path.basename(i)}",
                img[:, :round(w*0.4)]
            )
            cv2.imwrite(
                f"{os.path.join(output_folder, 'b/')}{os.path.basename(i)}",
                img[:, round(w*0.4):round(w*0.6)]
            )
            cv2.imwrite(
                f"{os.path.join(output_folder, 'c/')}{os.path.basename(i)}",
                img[:, round(w*0.6):]
            )
    
    else:
        print("❌ Choose between 'plt' or 'cv2' for method")
        return


#### Run:

In [0]:
split_images(input_folder='../predictions/', output_folder='../predictions/', method='plt')

In [0]:
%ls ../predictions